In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import csv

# 指定 CSV 文件名
filename = "records.csv"
# 開啟文件進行追加
csvfile = open(filename, 'a', newline='')
# 定義列標題
fieldnames = ['channel_name', 'video_title', 'views', 'comments', 'duration', 'likes', 'publish_date', 'category']
# 建立 writer
writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
# writer.writeheader() # 若是新的檔案，要加列標題
csvfile.flush()

# set start index for writing csv based on number of records.
d = pd.read_csv(filename)
start_idx = len(d)
del d

df = pd.read_csv("video_links_1.csv", encoding = "utf8", usecols = ['video_link',])
links = df.video_link

options = Options()
options.chrome_executable_path = (
    "/Users/chenxingchun/Python_彭彭/youtuber analysis/chromedriver.exe"
)

options.add_extension(
    "/Users/chenxingchun/Library/Application Support/Google/Chrome/Default/Extensions/gighmmpiobklfepjocnamgkkbiglidom/5.13.0_0.crx"
)
driver = webdriver.Chrome(options=options)

channel = []
title = []
publish_date = []
view_count = []
likes = []
comment_count = []
duration = []
categories = []

for idx in range(start_idx, len(links)):
    link = links[idx]
    print(idx, len(links), "is running, have not saved")
    driver.get(link)
    driver.refresh()
    # channel name
    time.sleep(10)

    channel.append(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ".//a[@class='yt-simple-endpoint style-scope yt-formatted-string']"))
        ).text)

    title.append(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//h1//yt-formatted-string[@class='style-scope ytd-watch-metadata']"))
        ).text)

    views = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@id="info-container"]/yt-formatted-string/span[1]'))
        ).text
    views = views.replace(" views", "")
    view_count.append(views)
    # comment count
    driver.execute_script("window.scroll(0, 500)")
    time.sleep(2)
    comment_count.append(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h2[@id="count"]/yt-formatted-string/span[1]'))
        ).text)

    yt_initial_data = driver.execute_script("return window['ytInitialData'];")
    yt_initial_player_response = driver.execute_script(
        "return window['ytInitialPlayerResponse'];"
    )

    for content in yt_initial_data['contents']['twoColumnWatchNextResults']['results']['results']['contents']:
        if "videoPrimaryInfoRenderer" in content.keys():
            like_count = content['videoPrimaryInfoRenderer']['videoActions']['menuRenderer']['topLevelButtons'][0]['segmentedLikeDislikeButtonRenderer']['likeButton']['toggleButtonRenderer']['defaultText']['simpleText']
    likes.append(like_count)
    # publish date
    date = yt_initial_player_response["microformat"]["playerMicroformatRenderer"][
        "publishDate"
    ]
    publish_date.append(date)
    # category
    category = yt_initial_player_response["microformat"]["playerMicroformatRenderer"][
        "category"
    ]
    categories.append(category)
    #duration
    length = yt_initial_player_response['videoDetails']['lengthSeconds']
    duration.append(length)

    record = {
        "channel_name": channel[0],
        "video_title": title[0],
        "views": view_count[0],
        "comments": comment_count[0],
        "duration": duration[0],
        "likes": likes[0],
        "publish_date": publish_date[0],
        "category": categories[0],
    }
    writer.writerow(record)
    csvfile.flush()

    channel = []
    title = []
    publish_date = []
    view_count = []
    likes = []
    comment_count = []
    duration = []
    categories = []

csvfile.close()